In [ ]:
import json
import os
import sys
import time
from typing import Iterable, List, Dict, Optional
import requests


OPENROUTER_API_URL = "https://openrouter.ai/api/v1/chat/completions"
DEFAULT_MODEL = "meta-llama/llama-3.3-8b-instruct:free"

class OpenRouterError(Exception):
    pass

In [ ]:
# Fix: add comma after 'location: str' and before 'num_children: int'
def build_messages(
    num_children: int,
    ages: List[int],
    weather: str, 
    location: str, 
    special_cases: str, 
) -> List[dict]:
    
    messages: List[dict] = []
    system_prompt = """You are an expert in pedagogy, child development, and educational activity design. 

**Your role:** Generate safe, fun, and age-appropriate activities for children, considering their ages, the number of children, location, weather, and any special cases.

# Guidelines:
- Suggest 3 indoor and 3 outdoor activities (depending on weather and location).
- Adapt to the age group(s) of the children.
- Outdoor activities must be realistic and possible near the provided location, given the weather.
- Check the provided location and sugggest outdoor activities according to the natural specifics of it. 
- Respect special cases (e.g., allergy, disability, space limitation).
- Provide 3–4 activity ideas.
- For each activity, include:
  - **Title with an icon/emoji**
  - **Description (3-4 sentences)**
  - **Why it’s good for the children** (developmental or educational value).
- Keep the tone practical, friendly, and parent-oriented.
- If the special cases are not related to children's activities, you should say "I'm sorry, I can only help with children's activities."
- If the special cases are not related to the children's ages, the weather or the location, you should say "I'm sorry, I can only help with children's activities."
- DO NOT write anything sexual, violent, or inappropriate for children.
"""
    user_prompt = f"""
Number of children: {num_children}
Ages of children: {ages}
Location: {location}
Weather: {weather}
Special case: {special_cases}

Please suggest suitable activities based on these inputs.
"""

    messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    return messages

def get_inputs():
    try:
        num_children = int(input("Enter the number of children: "))
    except ValueError:
        num_children = 1
    try:
        ages_input = input("Enter the ages of the children separated by ',' (ex.: 4,5,6): ")
        ages = [int(i.strip()) for i in ages_input.split(",") if i.strip()]
    except Exception:
        ages = [4]
    weather = input("How is the weather? (sunny, rainy, snowy): ") or "sunny"
    location = input("Enter your location: ") or "Yverdon-les-Bains"
    special_cases = input("Mention any special cases if you have any: ") or "No special case."
    
    prompts = build_messages(
        num_children=num_children,
        ages=ages,
        weather=weather,
        location=location,
        special_cases=special_cases if special_cases else None
    )
    return prompts

In [36]:
def chat_completion(
    prompts: dict,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None,
    top_p: Optional[float] = None,
    request_timeout: int = 120,
    metadata_app_name: Optional[str] = None,
    retries: int = 2,
) -> str:
    """
    Call OpenRouter chat completions and return the full response text
    (non-stream) or print streamed chunks and return the accumulated text.
    """

    api_key = None
    try:
        with open(".env", "r") as env_file:
            for line in env_file:
                if line.strip().startswith("OPENROUTER_API_KEY="):
                    api_key = line.strip().split("=", 1)[1]
                    break
    except FileNotFoundError:
        api_key = None
        
    if not api_key:
        raise OpenRouterError(
            "Missing OPENROUTER_API_KEY environment variable. Get a key from https://openrouter.ai"
        )

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    # Optional routing metadata helps with analytics in OpenRouter dashboard
    if metadata_app_name:
        headers["HTTP-Referer"] = metadata_app_name
        headers["X-Title"] = metadata_app_name

    payload: dict = {
        "model": model,
        "messages": prompts,
        "temperature": temperature,
    }
    if max_tokens is not None:
        payload["max_tokens"] = max_tokens
    if top_p is not None:
        payload["top_p"] = top_p

    # Basic retry with exponential backoff
    last_error: Optional[Exception] = None
    for attempt in range(retries + 1):
        try:
            return send_request(headers, payload, request_timeout)
        except (requests.HTTPError, requests.ConnectionError, requests.Timeout) as e:
            last_error = e
            if attempt < retries:
                sleep_s = 2 ** attempt
                time.sleep(sleep_s)
                continue
            break

    raise OpenRouterError(f"Request failed after {retries + 1} attempts: {last_error}")


def send_request(headers: dict, payload: dict, request_timeout: int) -> str:
    response = requests.post(
        OPENROUTER_API_URL, headers=headers, data=json.dumps(payload), timeout=request_timeout
    )
    _raise_for_bad_status(response)
    data = response.json()
    try:
        return data["choices"][0]["message"]["content"]
    except (KeyError, IndexError) as e:
        
        # REMOVE THIS
        print(data)
        raise OpenRouterError(f"Unexpected response format: {data}") from e

def _raise_for_bad_status(response: requests.Response) -> None:
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        # Try to include json error details if present
        try:
            details = response.json()
        except Exception:
            details = response.text
        raise OpenRouterError(f"HTTP {response.status_code}: {details}") from e

In [38]:
messages = get_inputs()
result = chat_completion(prompts=messages, model="x-ai/grok-4-fast:free")
result

'### Fun Activities for Your 3- and 4-Year-Olds in Lausanne\n\nWith two little ones aged 3 and 4, and sunny weather in Lausanne, I\'ve focused on simple, engaging activities that build on their curiosity and energy. These are perfect for siblings to play together, fostering cooperation and imagination. I\'ve included two indoor options for any downtime and two outdoor ones that take advantage of Lausanne\'s beautiful parks and lakeside spots.\n\n#### 🏠 Indoor Activity 1: Sensory Play Dough Fun\nCreate homemade play dough using flour, salt, water, and food coloring, then let the kids shape animals or pretend food on a kitchen table. Provide safe tools like cookie cutters and rolling pins for them to explore textures and forms.  \n**Why it\'s good:** This boosts fine motor skills and creativity, helping preschoolers practice hand-eye coordination while encouraging imaginative play that supports early language development through describing their creations.\n\n#### 📚 Indoor Activity 2: St

In [39]:
print(result)

### Fun Activities for Your 3- and 4-Year-Olds in Lausanne

With two little ones aged 3 and 4, and sunny weather in Lausanne, I've focused on simple, engaging activities that build on their curiosity and energy. These are perfect for siblings to play together, fostering cooperation and imagination. I've included two indoor options for any downtime and two outdoor ones that take advantage of Lausanne's beautiful parks and lakeside spots.

#### 🏠 Indoor Activity 1: Sensory Play Dough Fun
Create homemade play dough using flour, salt, water, and food coloring, then let the kids shape animals or pretend food on a kitchen table. Provide safe tools like cookie cutters and rolling pins for them to explore textures and forms.  
**Why it's good:** This boosts fine motor skills and creativity, helping preschoolers practice hand-eye coordination while encouraging imaginative play that supports early language development through describing their creations.

#### 📚 Indoor Activity 2: Storytime Puppe

In [ ]:
def chat_completion(
    prompt: str,
    system: Optional[str] = None,
    model: str = DEFAULT_MODEL,
    temperature: float = 0.7,
    max_tokens: Optional[int] = None,
    top_p: Optional[float] = None,
    request_timeout: int = 120,
    metadata_app_name: Optional[str] = None,
    retries: int = 2,
) -> str:
    """
    Call OpenRouter chat completions and return the full response text
    (non-stream) or print streamed chunks and return the accumulated text.
    """

    api_key = os.getenv("OPENROUTER_API_KEY")
    if not api_key:
        raise OpenRouterError(
            "Missing OPENROUTER_API_KEY environment variable. Get a key from https://openrouter.ai"
        )

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }

    # Optional routing metadata helps with analytics in OpenRouter dashboard
    if metadata_app_name:
        headers["HTTP-Referer"] = metadata_app_name
        headers["X-Title"] = metadata_app_name

    payload: dict = {
        "model": model,
        "messages": build_messages(prompt, system),
        "temperature": temperature,
        "stream": stream,
    }
    if max_tokens is not None:
        payload["max_tokens"] = max_tokens
    if top_p is not None:
        payload["top_p"] = top_p

    # Basic retry with exponential backoff
    last_error: Optional[Exception] = None
    for attempt in range(retries + 1):
        try:
            return _non_streaming_request(headers, payload, request_timeout)
        except (requests.HTTPError, requests.ConnectionError, requests.Timeout) as e:
            last_error = e
            if attempt < retries:
                sleep_s = 2 ** attempt
                time.sleep(sleep_s)
                continue
            break

    raise OpenRouterError(f"Request failed after {retries + 1} attempts: {last_error}")


def _non_streaming_request(headers: dict, payload: dict, request_timeout: int) -> str:
    response = requests.post(
        OPENROUTER_API_URL, headers=headers, data=json.dumps(payload), timeout=request_timeout
    )
    _raise_for_bad_status(response)
    data = response.json()
    # REMOVE THIS
    #print(data)
    try:
        return data["choices"][0]["message"]["content"]
    except (KeyError, IndexError) as e:
        raise OpenRouterError(f"Unexpected response format: {data}") from e

def _raise_for_bad_status(response: requests.Response) -> None:
    try:
        response.raise_for_status()
    except requests.HTTPError as e:
        # Try to include json error details if present
        try:
            details = response.json()
        except Exception:
            details = response.text
        raise OpenRouterError(f"HTTP {response.status_code}: {details}") from e


def get_inputs(num_children: int = 1, 
               ages: list = [3], 
               weather: str = "sunny", 
               location: str = "Yverdon-les-Bains") -> dict:

    num_children = input("Enter the number of children:")
    ages = list(input("Enter the ages of the children by seperating with ','(ex.: 4,5,6):").split())
    weather = input("How is the weather? (sunny, rainy, snowy)")
    location = input("Enter your location:")
    special_cases = input("Mention about the special cases if you have any:")
    
    prompt = build_messages(num_children=num_children, ages=ages, weather=weather, location=location, special_cases=special_cases)
    print(prompt)

    parser = argparse.ArgumentParser(description="OpenRouter Qwen 3-8B Instruct client")
    parser.add_argument(
        "--prompt",
        required=True,
        help="User prompt/message to send to the model",
    )
    
    parser.add_argument(
        "--model",
        default=DEFAULT_MODEL,
        help="Model name (see https://openrouter.ai/models)",
    )
    parser.add_argument(
        "--temperature",
        type=float,
        default=0.7,
        help="Sampling temperature",
    )
    
    parser.add_argument(
        "--ages",
        default=3,
        help="Optional system instruction (assistant behavior)",
    )

    parser.add_argument(
        "--top-p",
        type=float,
        default=None,
        help="Nucleus sampling probability",
    )
    parser.add_argument(
        "--max-tokens",
        type=int,
        default=None,
        help="Max tokens to generate",
    )
    parser.add_argument(
        "--stream",
        action="store_true",
        help="Stream tokens as they are generated",
    )
    parser.add_argument(
        "--timeout",
        type=int,
        default=120,
        help="Request timeout in seconds",
    )
    parser.add_argument(
        "--app-name",
        type=str,
        default=None,
        help="Optional app name for OpenRouter routing metadata",
    )
    
    return parser.parse_args(argv)


def main(argv: Optional[List[str]] = None) -> int:
    args = parse_args(argv)
    try:
        text = chat_completion(
            prompt=args.prompt,
            system=args.system,
            model=args.model,
            temperature=args.temperature,
            max_tokens=args.max_tokens,
            top_p=args.top_p,
            stream=args.stream,
            request_timeout=args.timeout,
            metadata_app_name=args.app_name,
        )
        if not args.stream:
            print(text)
        return 0
    except OpenRouterError as e:
        print(f"Error: {e}", file=sys.stderr)
        return 1


if __name__ == "__main__":
    raise SystemExit(main())




NameError: name 'dfsdfgsef' is not defined